!pip install mediapipe opencv-python
!pip install protobuf==3.20.

In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:

#to get realtime video feed from webcam
cap = cv2.VideoCapture(0) #setting up video capture device (number 0 represents webcam)
#while we are capturing
while cap.isOpened():
    #ret is return variable, frame will give us the image from webcam
    ret,frame = cap.read()
    #cv2.imshow gives us the pop on the screen which will ask to visualise a particular image
    #we pass the frame name as 'MediaPipe Feed' here in the brackets 
    cv2.imshow('MediaPipe Feed',frame)
    
    #cv2 waitkey() allows you to wait for a specific time in milliseconds until you press any button on the keyword.
    #press q to exit the window
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


## Make Detections

In [ ]:
cap = cv2.VideoCapture(0)
#setup mediapipe instance
#detection confidence and tracking confience
#setting up really high detection would only show results when clearly body is visible and setting it up too low will not give us accuarte results
#So setting it 0.5 works pretty well(50%)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        #Detect stuff and render
        
        #the feed is by default in the format of BGR color arrays
        #Recolor image - we do this because when pass our imagein media pipe we need it in RGB format
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) #reordering the colors to RGB
        image.flags.writeable = False #setting writebale false, we are basiccaly gonna save a bunch of memory once we pass this to pose estimation model
        
        #Make detection and storing in results
        results = pose.process(image)
        
        #image writeable true and again recolor the image to BGR(feed/prev format)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2), #drawing_sepcs = drawing specifications
                                 mp_drawing.DrawingSpec(color=(245,66,130),thickness=2,circle_radius=2)
                                 ) #drawing_sepcs = drawing specifications
        
        #print(results)
        
        cv2.imshow('MediaPipe Feed', image) #changed frame to image to pass the image labelled above

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# mp_drawing.draw_landmarks??

In [ ]:
# results.pose_landmarks

In [ ]:
# mp_pose.POSE_CONNECTIONS

## Determining joints

![Image](Image.jpg) 

In [ ]:
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 
        image.flags.writeable = False 
        

        results = pose.process(image)
        
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks 
        # If we are not able to make any specific detections, we can just pass instead of destroying the entire feed
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2), #drawing_sepcs = drawing specifications
                                 mp_drawing.DrawingSpec(color=(245,66,130),thickness=2,circle_radius=2)
                                 ) #drawing_sepcs = drawing specifications
        
        
        cv2.imshow('MediaPipe Feed', image) #changed frame to image to pass the image labelled above

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
landmarks

In [ ]:
len(landmarks) #same as the markdown image above

In [ ]:
for lm in mp_pose.PoseLandmark: #lm is landmark
    print(lm)

In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

In [ ]:
mp_pose.PoseLandmark.LEFT_SHOULDER.value

In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x

In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y

In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility

In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

## Calculating Angles

In [ ]:
#if you are calculating something that ranges between -90 and 90 degrees like latitude, use arctan. If calculating an angle that can be between -180 and 180 degrees, use arctan2.
def calculateAngle(a,b,c):
    a = np.array(a) #first
    b = np.array(b) #mid
    c = np.array(c) #end
    
    radians = np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle>180:
        angle = 360 - angle
    
    return angle

In [ ]:
#these are the coordinates based on the size of the image
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

In [ ]:
shoulder, elbow, wrist

In [ ]:
calculateAngle(shoulder, elbow, wrist) #calculates the angle

In [ ]:
np.multiply(elbow, [640,480]).astype(int) #this multiplication gives us the coordinates wrt to the dimensions of the webcam

In [ ]:

"""
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 
        image.flags.writeable = False 
        

        results = pose.process(image)
        
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks 
        # If we are not able to make any specific detections, we can just pass instead of destroying the entire feed
        try:
            landmarks = results.pose_landmarks.landmark
            #Get cooridnates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            #Calculate angle
            angle = calculateAngle(shoulder, elbow, wrist)
            
            #Visualise 
            #[640,480] are the dimensions of the webcam 
            #We are using elbow as the angle is calculated at the elbow
            cv2.putText(image, str(angle),
                            tuple(np.multiply(elbow, [640,480]).astype(int),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255),2, cv2.LINE_AA)
                       )
            print(landmarks)
        except:
            pass

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2), #drawing_sepcs = drawing specifications
                                 mp_drawing.DrawingSpec(color=(245,66,130),thickness=2,circle_radius=2)
                                 ) #drawing_sepcs = drawing specifications
        
        
        cv2.imshow('MediaPipe Feed', image) #changed frame to image to pass the image labelled above

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
"""

## Curl counter

In [ ]:
cap = cv2.VideoCapture(0)

counter = 0
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            #curl counter
            if angle>160:
                stage="down"
            if angle<30 and stage=='down': #come from the down position to up
                stage='up'
                counter+=1
                print(counter)
                       
        except:
            pass
        
        #Render curl counter
        #Setup status box
        cv2.rectangle(image,(0,0),(225,73),(245,117,16),-1) #-1 makes it a color box
        
        #Rep data
        #cv2.putText(image, data, startcoordinate, font, size of text, colour,linewidth, linetype )
        cv2.putText(image, 'REPS', (15,12),
                   cv2.FONT_HERSHEY_SIMPLEX,0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter) , (10,60),
                   cv2.FONT_HERSHEY_SIMPLEX,2, (255,255,255), 2, cv2.LINE_AA)
        
        #Stage data
        cv2.putText(image, 'STAGE', (65,12),
                   cv2.FONT_HERSHEY_SIMPLEX,0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage , (65,60),
                   cv2.FONT_HERSHEY_SIMPLEX,2, (255,255,255), 2, cv2.LINE_AA)
        
        #Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,16), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()